<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/6_TimeSeriesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prerequisite for this notebook is:
>[5: TimeSeriesForecasting.ipynb](https://colab.research.google.com/drive/11B5w7z3oGqgeDi_6mGe8aKTKcuO6P4x3#scrollTo=eiZsm3AuTNwx)

In this notebook you look at furniture and office supply sales over a four year period. Then you create models to forecast future sales.<br>

https://algorithmia.com/blog/an-introduction-to-time-series-forecasting

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/timeSeries.git cloned-repo
%cd cloned-repo

**Import libraries**

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib

In [ ]:
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

# **Get and prepare the data**<br>
It is in the form of an Excel spreadsheet, use Pandas to read in the data. <br>
The data is from a Superstore that sells furniture and office supplies.

In [ ]:
df = pd.read_excel("Sample - Superstore.xls")

In [ ]:
df.columns

In [ ]:
df.head()

Save the rows on furniture sales in a variable called furniture

In [ ]:
furniture = df.loc[df['Category'] == 'Furniture']

The data spans the time Jan.,2014 to Dec.,2017

In [ ]:
furniture['Order Date'].min(), furniture['Order Date'].max()

Check for missing data and only keep the Order Date and Sales columns

In [ ]:
cols = ['Row ID', 'Order ID', 'Ship Date', 'Ship Mode', 'Customer ID', 
        'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code',
        'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 
        'Quantity', 'Discount', 'Profit']
furniture.drop(cols, axis=1, inplace=True)
furniture = furniture.sort_values('Order Date')
print("Null sums: ",furniture.isnull().sum())

In [ ]:
furniture

Put the data into order of earliest date to latest date. <br>
Reset the index so the earliest date is index 0

In [ ]:
furniture = furniture.groupby('Order Date')['Sales'].sum().reset_index()

In [ ]:
furniture

Change the index from row number to the date. <br>

In [ ]:
furniture = furniture.set_index('Order Date')
furniture.index

The data is now two columns, order data and Sales<br>
It is ordered from the earliest date to the latest date.<br>
Sales for every day are recorded.

In [ ]:
furniture

Group the sales into months by adding all the sales for each month together. The date for the month will be the first day of the month

In [ ]:
#y = furniture['Sales'].resample('MS').mean()
monthlySales = furniture['Sales'].resample('MS').mean()

In [ ]:
print(monthlySales['2016'])
print("-------End 2016--------------")
print(monthlySales['2017'])
print("----------End 2017-----------")

Plot the monthly sales for furniture

In [ ]:
monthlySales.plot(figsize=(15, 6))
plt.show()

# **Decompose the data**

Using time series decomposition, we can decompose the time series into three components: <br>
- trend<br>
- seasonality<br>
- noise (residual)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8
decomposition = sm.tsa.seasonal_decompose(monthlySales, model='additive')
fig = decomposition.plot()
fig.suptitle("Montly Sales Decomposed")
plt.show()


# **ARIMA (Autoregressive Integrated Moving Average)**<br>
ARIMA models are denoted with the notation ARIMA(p, d, q). These three parameters account for seasonality, trend, and noise in data. <br><br>
p,d,q are given a range of values 0,1,2. <br>
These ranges are used to do a grid search to find the optimal pdq parameters for the model and data. <br>
The model is run with each combination of pdq.<br>
The best performing combination (the one with the lowest AIC) is kept. 


ARIMA is actually a class of models that ‘explains’ a given time series based on its own past values, that is, its own lags and the lagged forecast errors, so that equation can be used to forecast future values.

Any ‘non-seasonal’ time series that exhibits patterns and is not a random white noise can be modeled with ARIMA models.

An ARIMA model is characterized by 3 terms: p, d, q

where,

p is the order of the AR term

q is the order of the MA term

d is the number of differencing required to make the time series stationary



If a time series, has seasonal patterns, then you need to add seasonal terms and it becomes SARIMA, short for ‘Seasonal ARIMA’. 

# **Seasonal-ARIMA (SARIMA)**<br>
[SARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html)<br>
<br>
SARIMAX is much like ARIMA, but a little more complicated. Not only do you have to use a loop and grid search for the optimal values of p, d, and q, but you have to also use a nested loop and grid search for the seasonal values for p, d, and q.
<br>


The (P,D,Q,s) order of the seasonal component of the model for the AR parameters, differences, MA parameters, and periodicity.<br>

s is an integer giving the periodicity (number of periods in season), often it is 4 for quarterly data or 12 for monthly data. Default is no seasonal effect.

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
#12 is for monthly data
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

**Find the best p,d,q combination**

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(monthlySales,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = model.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

**The Akaike Information Critera (AIC)** is a widely used measure of a statistical model. <br>
It basically quantifies<br>
1) the goodness of fit<br>
2) the simplicity/parsimony, of the model into a single statistic.

**When comparing two models, the one with the lower AIC is generally “better”**

The above output suggests that SARIMAX(1, 1, 1)x(1, 1, 0, 12) yields the lowest AIC value of 297.78. Therefore we should consider this to be optimal option.


# **Train the Model**<br>
The choosen model is trained on the data<br>
- order: (p,d,q) order of the model for the number of AR parameters, differences, and MA parameters
- seasonal_order: (P,D,Q,s) order of the seasonal component of the model for the AR parameters, differences, MA parameters, and periodicity
- enforce_stationarity: Whether or not to transform the AR parameters to enforce stationarity in the autoregressive component of the model
- [enforce_invertibility](https://medium.com/@kfoofw/arma-causality-and-invertibility-of-stationary-time-series-971b7d87b79c): Whether or not to transform the MA parameters to enforce invertibility in the moving average component of the model

In [ ]:
modelTrained = sm.tsa.statespace.SARIMAX(monthlySales,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = modelTrained.fit()
print(results.summary().tables[1])

**The model summary above** contains the ‘Covariance Type’ graph, which depicts each of the variables’ impact on the forecast. 

We have three main lagged AR and MA variables. 

- The first set of AR and MA variables is lagged by 1 time step (ar.L1 and ma.L1, respectively), 

- The second ar is lagged by 12 time steps (ar.S.L12).
<br>
Looking at the ‘P>abs(z)’ term in the graph we want our P values to be as close to 0 as possible.<br> Using a cutoff of <0.05 for statistical significance, our MA term significantly impacts model forecast.

Now, run the model diagnostics. It will hopefully show any unusal behavior. <br>
In this case, our model diagnostics suggests that the model residuals are near normally distributed.

**Residuals**<br>
**Residual is the difference between true and predicted value**. <br>
If there are correlations between residuals - there is information left in the residuals which should be used in computing forecasts. If the residuals have a mean other than zero, then the forecasts are biased. For instance if we have a constantly growing residual like (... -0.3, -0.2, 0.1, 0, 0.1, 0.2, 0.3, ... and so on, the mean will be 0) it means that our model does not fully depict the process.

**Histogram** plus estimated density:  of standardized residuals, along with a Normal(0,1) density plotted for reference. <br><br>
[Kernel density estimation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.plot.kde.html) (KDE) is a non-parametric way to estimate the probability density function (PDF) of a random variable. 

**Normal Q-Q plot, with Normal reference line**<br><br>
**Normal Q–Q (quantile-quantile) plot** is a probability plot, which is a graphical method for comparing two probability distributions by plotting their quantiles against each other.

In [ ]:
from IPython.display import Image
Image("normalQQPlot.png" , width=640)

**Correlogram**: correlogram is a commonly used tool for checking randomness in a data set. <br>
**If random, autocorrelations should be near zero** for any and all time-lag separations. <br><br>
If non-random, then one or more of the autocorrelations will be significantly non-zero.<br>

In addition, correlograms are used in the model identification stage for Box–Jenkins autoregressive moving average time series models. **Autocorrelations should be near-zero for randomness**; *if the analyst does not check for randomness, then the validity of many of the statistical conclusions becomes suspect*. The correlogram is an excellent way of checking for such randomness.

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.show()

**Making predictions with the model**

To understand the accuracy of our forecasts, we compare predicted sales to real sales of the time series, and we set forecasts to start at 2017–01–01 to the end of the data.

The line plot is showing the observed values compared to the rolling forecast predictions. <br><br>
Overall, our forecasts align with the true values very well, showing an upward trend starts from the beginning of the year and captured the seasonality toward the end of the year.

In [ ]:
#Assignment 1 - change the date here
pred = results.get_prediction(start=pd.to_datetime('2017-01-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = monthlySales['2014':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

The MSE is a measure of the quality of an estimator — it is always non-negative, and the smaller the MSE, the closer we are to finding the line of best fit.

In [ ]:
monthlySales_forecasted = pred.predicted_mean
monthlySales_truth = monthlySales['2017-01-01':]
mse = ((monthlySales_forecasted - monthlySales_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

Root Mean Square Error (RMSE) tells us that our model was able to forecast the average daily furniture sales in the test set within 151.64 of the real sales. <br><br>
Our furniture daily sales range from around 400 to over 1200. This is a pretty good model so far.

In [ ]:
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

**Producing and visualizing forecasts**

The model clearly captures furniture sales seasonality. <br><br>
As we forecast further out into the future, it is natural for us to become less confident in our values. This is reflected by the confidence intervals generated by our model, which grow larger as we move further out into the future.

In [ ]:
pred_uc = results.get_forecast(steps=100)
pred_ci = pred_uc.conf_int()
ax = monthlySales.plot(label='observed', figsize=(14, 7))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

# **Assignment 1**<br>
Change the date in the predictions code. <br>
Chage the date from 2017-01-01 to 2016-01-01

What happens to the prediction? 
Why?

# **Assignment 2**<br>
We used the grid search method to find the best combinatiomn of p,d,q<br>
Now go back and pick a different combination of p,d,q. It can be the second best, the worst, or some combination in between. <br>
Compare the changes in the prediction between the runs. Try several combinations. 